**Task 1: Pre-process the dataset**

All the imports:

In [13]:
import csv
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

**Check how many patients/variables there are**

In [14]:
df = pd.read_csv("dataset_mood_smartphone.csv")
counted = df.groupby(["id"]).size()
print (len(counted))
print (counted)

27
id
AS14.01    21999
AS14.02    14581
AS14.03    14425
AS14.05    15745
AS14.06    18092
AS14.07    16045
AS14.08     7902
AS14.09    10886
AS14.12    17311
AS14.13    19592
AS14.14     9286
AS14.15     2848
AS14.16     3982
AS14.17    15826
AS14.19    11397
AS14.20     3620
AS14.23    21852
AS14.24    14430
AS14.25    12589
AS14.26    16403
AS14.27    14575
AS14.28    19276
AS14.29    17499
AS14.30    17279
AS14.31    11889
AS14.32    11193
AS14.33    16390
dtype: int64


In [15]:
counted = df.groupby(["variable"]).size()
print (len(counted))
print (counted)

19
variable
activity                22965
appCat.builtin          91288
appCat.communication    74276
appCat.entertainment    27125
appCat.finance            939
appCat.game               813
appCat.office            5642
appCat.other             7650
appCat.social           19145
appCat.travel            2846
appCat.unknown            939
appCat.utilities         2487
appCat.weather            255
call                     5239
circumplex.arousal       5643
circumplex.valence       5643
mood                     5641
screen                  96578
sms                      1798
dtype: int64


There are __***27 patients***__, names from 01 to 33 (some numbers are missing)

There are __***19 variables***__



**Making individual files for each patient:**

In [16]:
RawFile = "dataset_mood_smartphone.csv"

with open(RawFile) as infile:
    reader = csv.reader(infile)
    #input_header = reader.next()
    for row in reader:
        patient = row[1]
        CleanedFile = "./data/" + patient +".csv"
        with open(CleanedFile, 'a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(row[2:5])

With good old-fashioned CTRL-F method we discover that: (AND REALIZING YOU DIDN'T NEED TO CHECK THAT CAUSE ITS ALL IN THE ASSIGNMENT DESCRIPTION)

activity                Numerical value - use mean
appCat.builtin          Time - add times
appCat.communication    Time - add times
appCat.entertainment    Time - add times
appCat.finance          Time - add times
appCat.game             Time - add times
appCat.office           Time - add times
appCat.other            Time - add times
appCat.social           Time - add times
appCat.travel           Time - add times
appCat.unknown          Time - add times
appCat.utilities        Time - add times
appCat.weather          Time - add times
call                    Numerical value - use mean
circumplex.arousal      Numerical value - use mean
circumplex.valence      Numerical value - use mean
mood                    Numerical value - use mean
screen                  Time
SMS                     Numerical values - use mean

Certain values have N/A so we need to delete that row.


In [29]:
name1 = "AS14.01.csv"
pathin = "./data/"+ name1
pathout = "./data/clean/" + name1
df = pd.read_csv(pathin)
#df
df = df[df.value.astype(str)  != 'NA']
df.to_csv(pathout, encoding='utf-8', index=False)